In [44]:
from astrosite_dataset import LabelledEventsAstrositeDataset

dataset_path = "data/astrosite/recordings"

dataset = LabelledEventsAstrositeDataset(dataset_path, split="all")
len(dataset)

6987

## Calculate spectrograms from 30s recordings

In [45]:
import numpy as np
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm

sample_counts = defaultdict(int)
save_to_folder = "data/astrosite/spectrograms"

for sample in tqdm(dataset):
    events, target_id = sample
    # check for ambiguous satellite
    labels = sorted(list(set(events["label"])))
    if labels[0] < -1:
        # print(f"Skipping sample with labels {labels}")
        continue
    array = np.zeros(30000)
    events = events[events["label"] == -1]
    events = events[events["t"] < 3e7]
    if len(events) < 1000:
        continue
    event_indices = (events["t"] // 1e3).astype(np.int64)
    np.add.at(array, event_indices, 1)
    nperseg = 16
    frequencies, times, Sxx = spectrogram(
        array, fs=1, nperseg=nperseg, noverlap=nperseg // 2
    )

    # Set the figure size dynamically based on the number of frequency bands
    fig_height = len(frequencies) / 10  # Adjust the divisor to control height scaling
    plt.figure(figsize=(10, fig_height))

    plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx + 1e-6), shading="auto")
    plt.axis("off")

    file_path = Path(f"{save_to_folder}/{target_id}/{sample_counts[target_id]}.png")
    file_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(file_path, bbox_inches="tight", pad_inches=0)
    plt.close()  # Close the figure to free memory

    sample_counts[target_id] += 1

print(sample_counts)

100%|██████████| 6987/6987 [07:17<00:00, 15.98it/s]

defaultdict(<class 'int'>, {'40697': 10, '39762': 13, '39358': 8, '43567': 6, '37765': 12, '43437': 18, '44076': 10, '23467': 21, '43873': 19, '49505': 21, '54377': 15, '53526': 6, '39059': 18, '54153': 2, '37834': 26, '51288': 2, '55264': 8, '39455': 12, '47719': 11, '52255': 17, '37846': 6, '27711': 29, '40146': 6, '55131': 24, '40982': 29, '52909': 8, '52984': 6, '54700': 15, '43920': 8, '47613': 19, '47510': 2, '43752': 22, '35500': 9, '40555': 5, '56371': 3, '39490': 7, '43753': 16, '36106': 2, '38733': 9, '44552': 28, '37168': 5, '50574': 39, '52944': 1, '39070': 24, '31800': 18, '39206': 27, '43618': 7, '44389': 6, '41471': 28, '40892': 34, '38098': 4, '26356': 11, '37237': 9, '44850': 9, '50005': 29, '47321': 27, '54214': 19, '44978': 10, '44637': 30, '29155': 4, '38356': 8, '50002': 10, '40922': 15, '54230': 28, '46497': 5, '40294': 15, '26038': 29, '55912': 20, '43657': 12, '53948': 6, '53946': 3, '37152': 12, '46805': 22, '42741': 28, '42969': 12, '53950': 6, '43874': 14, '4

In [46]:
import shutil
from pathlib import Path

save_to_folder = "data/astrosite/spectrograms"
min_samples = 30  # Set your minimum number of samples here

for target_id, count in sample_counts.items():
    if count < min_samples:
        folder_path = Path(f"{save_to_folder}/{target_id}")
        if folder_path.exists() and folder_path.is_dir():
            shutil.rmtree(folder_path)

print("Folders deleted.")

Folders deleted.


## Calculate time surfaces of 3s slices of the recordings

In [35]:
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from tonic import transforms
from itertools import islice


sample_counts = defaultdict(int)
save_to_folder = "data/astrosite/surfaces"

frame_time = 3e6
surface_decay = 2e6
surface_transform = transforms.ToTimesurface(
    sensor_size=dataset.sensor_size, dt=frame_time, tau=surface_decay
)

# for sample in tqdm(islice(dataset, 20)):
for sample in tqdm(dataset):
    events, target_id = sample
    # check for ambiguous satellite
    labels = sorted(list(set(events["label"])))
    if labels[0] < -1:
        continue
    events = events[events["label"] == -1]
    if len(events) < 2500:
        continue
    surfaces = surface_transform(events)

    for i, surface in enumerate(surfaces):
        file_path = Path(f"{save_to_folder}/{target_id}/{sample_counts[target_id]}.png")
        file_path.parent.mkdir(parents=True, exist_ok=True)
        plt.imsave(file_path, surface.sum(0), cmap="gray")
        sample_counts[target_id] += 1

print(sample_counts)

100%|██████████| 6987/6987 [17:39<00:00,  6.59it/s]

defaultdict(<class 'int'>, {'40697': 156, '39762': 196, '39358': 122, '43567': 46, '37765': 176, '43437': 258, '44076': 9, '49505': 183, '54377': 108, '53526': 76, '39059': 248, '37834': 216, '39455': 116, '47719': 89, '37846': 36, '40982': 190, '52909': 133, '52984': 28, '54700': 166, '43920': 64, '47510': 24, '43752': 355, '40555': 62, '56371': 18, '39490': 102, '43753': 245, '38733': 118, '23467': 37, '44552': 188, '37168': 66, '50574': 346, '52944': 15, '31800': 90, '39206': 244, '43618': 113, '41471': 101, '40892': 254, '26356': 18, '44850': 63, '47321': 209, '54214': 288, '44637': 250, '38356': 63, '50002': 34, '40922': 223, '54230': 243, '46497': 58, '55912': 138, '43657': 193, '53948': 90, '53946': 43, '37152': 184, '46805': 157, '42969': 178, '53950': 86, '50005': 200, '40261': 102, '54223': 88, '32393': 36, '37753': 18, '40286': 216, '41579': 110, '53761': 110, '41549': 114, '39761': 233, '33331': 73, '43058': 54, '42741': 181, '42063': 251, '45611': 18, '48914': 105, '44299'

In [43]:
import shutil
from pathlib import Path

save_to_folder = "data/astrosite/surfaces"
min_samples = 300  # Set your minimum number of samples here

for target_id, count in sample_counts.items():
    if count < min_samples:
        folder_path = Path(f"{save_to_folder}/{target_id}")
        if folder_path.exists() and folder_path.is_dir():
            shutil.rmtree(folder_path)

print("Folders deleted.")

Folders deleted.
